In [23]:
import pandas as pd

In [24]:
data_frame = pd.read_csv('data/pima-indians-diabetes.csv')
data_frame.columns

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'],
      dtype='object')

In [25]:
data_frame.shape

(768, 9)

View correlations

In [26]:
correlations = data_frame.corr(method='pearson')
print(correlations)

                          Pregnancies   Glucose  BloodPressure  SkinThickness  \
Pregnancies                  1.000000  0.129459       0.141282      -0.081672   
Glucose                      0.129459  1.000000       0.152590       0.057328   
BloodPressure                0.141282  0.152590       1.000000       0.207371   
SkinThickness               -0.081672  0.057328       0.207371       1.000000   
Insulin                     -0.073535  0.331357       0.088933       0.436783   
BMI                          0.017683  0.221071       0.281805       0.392573   
DiabetesPedigreeFunction    -0.033523  0.137337       0.041265       0.183928   
Age                          0.544341  0.263514       0.239528      -0.113970   
Outcome                      0.221898  0.466581       0.065068       0.074752   

                           Insulin       BMI  DiabetesPedigreeFunction  \
Pregnancies              -0.073535  0.017683                 -0.033523   
Glucose                   0.331357  0.221

In [27]:
from pandas_profiling import ProfileReport
pfr = ProfileReport(data_frame)
pfr

Number of variables,9
Number of observations,768
Total Missing (%),0.0%
Total size in memory,54.1 KiB
Average record size in memory,72.1 B
Numeric,8
Categorical,0
Boolean,1
Date,0
Text (Unique),0
Rejected,0


In [28]:
target_column='Outcome'
y = data_frame[target_column].values
features = data_frame.drop(target_column, axis=1)
X = features.values

In [29]:
TEST_SIZE = 0.3 # ratio of data to have in test
SEED = 8 # to be used to initialize random number generator, for reproducibility
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE, random_state=SEED)

In [30]:
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

FOLDS = 10
grid = {"max_depth": [3, 9, None],
        "max_features": [0.5, 0.75]}

kfold = KFold(n_splits=FOLDS, shuffle=True, random_state=SEED)
grid_search = GridSearchCV(RandomForestClassifier(random_state=SEED),
                           grid,
                           scoring="accuracy",
                           cv=kfold,
                           n_jobs=-1,
                           verbose=1)

In [31]:
grid_search.fit(X, y)
print(grid_search.best_params_)
clf = RandomForestClassifier(**grid_search.best_params_)

Fitting 10 folds for each of 6 candidates, totalling 60 fits
{'max_depth': 3, 'max_features': 0.5}


[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:    0.4s finished


In [32]:
%time clf.fit(X_train, y_train)
%time y_pred = clf.predict(X_test)

CPU times: user 13.2 ms, sys: 2.48 ms, total: 15.7 ms
Wall time: 13.6 ms
CPU times: user 1.39 ms, sys: 579 µs, total: 1.97 ms
Wall time: 1.46 ms


In [33]:
import sklearn.metrics as metrics

In [34]:
metrics.accuracy_score(y_test, y_pred)

0.7705627705627706

In [35]:
metrics.confusion_matrix(y_test, y_pred)

array([[133,  14],
       [ 39,  45]])

In [36]:
pd.crosstab(y_test, y_pred, rownames=['Actual'], colnames=['Predicted'])

Predicted,0,1
Actual,,
0,133,14
1,39,45


In [37]:
data_frame.columns

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'],
      dtype='object')

In [38]:
clf.feature_importances_

array([0.06324508, 0.55139314, 0.01395854, 0.03021586, 0.02726252,
       0.14521533, 0.06929188, 0.09941767])

In [39]:
clf.classes_

array([0, 1])

In [40]:
index = 230

In [41]:
X_test[index]

array([ 1.   , 88.   , 30.   , 42.   , 99.   , 55.   ,  0.496, 26.   ])

In [42]:
y_test[index]

1

In [43]:
clf.predict_proba([X_test[index]])

array([[0.86174226, 0.13825774]])

In [44]:
clf.predict([X_test[index]])

array([0])